Robot Dynamic Model Identification
=======
---

# 1 - Praparing work

## Import libraries

In [1]:
# enable auto-reload modules
%load_ext autoreload
%autoreload 2
#interactive plotting in separate window
%matplotlib qt 
#%matplotlib inline

from sympy import init_printing
init_printing()
    
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 8]

import numpy as np
import sympy
from robot_def import RobotDef
from kinematics import Geometry
from dynamics import Dynamics
from trajectory_optimization import TrajOptimizer
from trajectory_optimization import TrajPlotter
from utils import new_sym
from utils import save_data, load_data
import time

## Folder dir for saving and loading files

In [2]:
# model_name = 'test_psm_long2'
# model_name ='test'
# model_name = 'mtm_3links'
model_name = 'mtm'
# model_name = 'mtm_3links_parallel'
# model_name = 'mtm_4links_parallel'

model_folder = 'data/' + model_name + '/model/'

---
# 2 - Robot modelling

## Create joint variables and define their relations

## Robot geometry definition in following order

| Joint number | prev link | succ links | $a$ | $\alpha$ | $d$ | $\theta$ | link inertia | motor inertia | friction |
|--------------|-----------|------------|-----|----------|-----|----------|--------------|---------------|---------|

In [3]:
q0, q1, q2, q3, q4, q5, q6, q7, q8, q9, q10 = new_sym('q:11')
_pi = sympy.pi

dh = []
springs = []
friction_type = []

if model_name =='test':
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,8],  0,     0,      -0.21537, q1,       True,  False, False),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, False)]
    q = q1
    r_s = 0.0075
    h_s = 0.1035
    l_r = 0.067
    q_o = 23.0/180*_pi
    l = sympy.sqrt(r_s**2 + h_s**2 - 2*r_s*h_s*sympy.cos(_pi + q_o - q))
    d_l = l - l_r
    r_f = r_s * h_s * sympy.sin(_pi + q_o - q) / l
    tau_s = r_f * d_l # * 27.86
    springs = [tau_s]
    friction_type = ['Coulomb', 'viscous', 'offset']
elif model_name == 'mtm':
    # define paralelogram coordinate relation
    q3 = -q2 + q8
    q9 = -q8 + q2
    q10 = q8 - q2
    
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,8],  0,     0,      -0.21537, q1,       True,  False, True),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, True),
          (3,   2,  [4],    0.279, 0,      0,        q3+_pi/2, True,  False, True),
          (4,   3,  [5],    0.365, -_pi/2, 0.151,    q4,       True,  False, True),
          (5,   4,  [6],    0,     _pi/2,  0,        q5,       True,  False, True),
          (6,   5,  [7],    0,     -_pi/2, 0,        q6+_pi/2, True,  False, True),
          (7,   6,  [],     0,     -_pi/2, 0,        q7+_pi,   True,  False, True),
          (8,   1,  [9],    0,     -_pi/2, 0,        q8+_pi,   True,  False, True),
          (9,   8,  [10],   0.1,   0,      0,        q9-_pi/2, True,  False, True),
          (10,  9,  [],     0.279, 0,      0,        q10,      False, False, True)]

    q = q5
    r_s = 0.0075
    h_s = 0.1035
    l_r = 0.067
    q_o = 23.0/180.0*_pi
    l = sympy.sqrt(r_s**2 + h_s**2 - 2*r_s*h_s*sympy.cos(_pi + q_o - q))
    d_l = l - l_r
    r_f = r_s * h_s * sympy.sin(_pi + q_o - q) / l
    tau_s = r_f * d_l # * 27.86
    #springs = [tau_s]
    
    friction_type = ['Coulomb', 'viscous', 'offset']
    print("MTM")
elif model_name == 'mtm_3links_parallel':
    # define paralelogram coordinate relation
    q3 = -q2 + q4
    q5 = -q4 + q2
    q6 = q4 - q2
    
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,4],  0,     0,      -0.21537, q1,       True,  False, True),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, True),
          (3,   2,  [4],    0.279, 0,      0,        q3+_pi/2, True,  False, True),
          (4,   1,  [5],    0,     -_pi/2, 0,        q4+_pi,   True,  False, True),
          (5,   4,  [6],   0.1,   0,      0,        q5-_pi/2, True,  False, True),
          (6,   5,  [],     0.279, 0,      0,        q6,      False, False, True)]
    friction_type = ['Coulomb', 'viscous']
    #springs = [q1, q2, q3]
elif model_name == 'mtm_4links_parallel':
    # define paralelogram coordinate relation
    q3 = -q2 + q5
    q6 = -q5 + q2
    q7 = q5 - q2
    
    dh = [(0,   -1, [1],    0,     0,      0,        0,        False, False, False),
          (1,   0,  [2,5],  0,     0,      -0.21537, q1,       True,  False, True),
          (2,   1,  [3],    0,     -_pi/2, 0,        q2+_pi/2, True,  False, True),
          (3,   2,  [4],    0.279, 0,      0,        q3+_pi/2, True,  False, True),
          (4,   3,  [0],    0.365, -_pi/2, 0.151,    q4,       True,  False, True),
          (5,   1,  [6],     0,     -_pi/2, 0,        q5+_pi,   True,  False, True),
          (6,   5,  [7],   0.1,   0,      0,        q6-_pi/2, True,  False, True),
          (7,   6,  [],     0.279, 0,      0,        q7,      False, False, True)]
    friction_type = ['Coulomb', 'viscous']
elif model_name == 'mtm_3links':
    dh = [(0,   -1, [1],    0,      0,      0,          0,         False, False, False),
          (1,   0,  [2],    0,      0,      -0.21537,   q1,        True,  False, True),
          (2,   1,  [3],    0,      -_pi/2, 0,          q2+_pi/2,  True,  False, True),
          (3,   2,  [4],    0.279,  0,      0,          q3+_pi/2,  True,  True,  True)]
    # springs = [q1]
    friction_type = ['Coulomb', 'viscous']
elif (model_name == 'test_psm'):
    beta = 0.2908
    beta2 = 0.0767 + beta
    delta = 0.6025
    x = [None]*10
    x[0] = (0,   -1, [1],        0,              0,     0,                       0,      False,  False, False) # Base
    x[1] = (1,   0,  [2],        0,     sympy.pi/2,     0,           q1+sympy.pi/2,       True,  False,  True) # Yaw
    x[2] = (2,   1,  [3, 4],        0, -sympy.pi/2,     0,              sympy.pi/2,       False, False, False) # -- Intermediate
    x[3] = (3,   2,  [5, 6],  -0.0296,           0,     0,    q2-beta - sympy.pi/2,        True,  False, True) # Pitch Back
    x[4] = (4,   2,  [] ,     0.0664 ,           0,     0,    q2-beta - sympy.pi/2,        True,  False, True) # Pitch Front                                    )
    x[5] = (5,   2,  [7],     -0.0296,           0,     0,   q2-beta2 - sympy.pi/2,        False,  False, False)    # Intermediate
    x[6] = (6,   3,  [8],       0.150,           0,     0,    -q2+beta+sympy.pi/2,      True,  False,  True)    # Pitch Bottom
    x[7] = (7,   5,  [],       0.1842,           0,     0,    -q2+beta2+sympy.pi/2,     True,  False,  True)    # Pitch Top
    x[8] = (8,   6,  [9],       0.516,           0,     0,                      q2,     True,  False,  True)    # Pitch End
    x[9] = (9,   8,  [10],      0.043, -sympy.pi/2,-.28809+q3,          sympy.pi/2,     True,  False, True)    # Main Insertion
    dh = x
    friction_type = ['Coulomb', 'viscous', 'offset']              
elif (model_name == 'test_psm_long2'):
    beta = 0.2908
    beta2 = 0.0767 + beta
    delta = 0.6025
    
    x = [None]*17
    x[0]  = ( 0,  -1,  [1],          0,           0,     0,                       0, False, False, False) # Base
    x[1]  = ( 1,   0,  [2],          0,  sympy.pi/2,     0,           q1+sympy.pi/2,  True, False,  True) # Yaw
    x[2]  = ( 2,   1,  [3, 4, 5],       0, -sympy.pi/2,     0,              sympy.pi/2, False, False, False) # -- Intermediate
    x[3]  = ( 3,   2,  [6, 10], -0.0296,       0,     0,    q2-beta - sympy.pi/2,  True, False,  True) # Pitch Back
    x[4]  = ( 4,   2,  [] ,     0.0664,           0,     0,    q2-beta - sympy.pi/2,  True, False,  True) # Pitch Front                                    )
    x[5]  = ( 5,   2,  [7],    -0.0296,           0,     0,   q2-beta2 - sympy.pi/2, False, False, False)    # Intermediate
    x[6]  = ( 6,   3,  [8],      0.150,           0,     0,     -q2+beta+sympy.pi/2,  True, False,  True)    # Pitch Bottom
    x[7]  = ( 7,   5,   [],     0.1842,           0,     0,    -q2+beta2+sympy.pi/2,  True, False,  True)    # Pitch Top
    x[8]  = ( 8,   6,  [9],      0.516,           0,     0,                      q2,  True, False,  True)    # Pitch End
    x[9]  = ( 9,   8, [12],      0.043, -sympy.pi/2,-.28809+q3,          sympy.pi/2,  True, False,  True)    # Main Insertion
    x[10] = (10,   3, [11],          0,           0,     0,         beta+sympy.pi/2, False, False, False) # Intermediate Counterweight
    x[11] = (11,  10,   [],       -0.1, +sympy.pi/2, delta*q3,                    0,  True, False,  True) # Counterweight
    
    x[12] = (12,   9, [13],          0,           0,0.4162,          -sympy.pi/2+q4, False,  True,  True) # Tool Roll 
    x[13] = (13,  12, [14],          0, -sympy.pi/2,     0,          -sympy.pi/2+q5, False,  True,  True) # Tool Pitch
    x[14] = (14,  13, [15,16],  0.0091, -sympy.pi/2,     0,          -sympy.pi/2+q6, False,  True,  True) # Tool Yaw
    x[15] = (15,  14, [],            0,           0,     0,                     -q7, False, False,  True) # Gripper1
    x[16] = (16,  15, [],            0,           0,     0,                      q7, False,  True,  True) # Gripper2
    dh = x
    friction_type = ['Coulomb', 'viscous', 'offset']
else:
    raise Exception("No robot name matches " + name)
print(friction_type)
robot_def = RobotDef(dh,
                     springs=springs,
                     dh_convention='mdh',
                     friction_type=friction_type)

MTM
['Coulomb', 'viscous', 'offset']


In [72]:
robot_def.bary_params

## Create kinematics chain

In [73]:
geom = Geometry(robot_def)

In [74]:
angle = [0, 0, 0, 0, 0, 0, 0]
geom.draw_geom(angle)

## Create dynamics

In [75]:
%time dyn = Dynamics(robot_def, geom)

Calculating Lagrangian...
Calculating the link kinetic energy of 1/10
Calculating the link kinetic energy of 2/10
Calculating the link kinetic energy of 3/10
Calculating the link kinetic energy of 4/10
Calculating the link kinetic energy of 5/10
Calculating the link kinetic energy of 6/10
Calculating the link kinetic energy of 7/10
Calculating the link kinetic energy of 8/10
Calculating the link kinetic energy of 9/10
Calculating joint torques...
tau of q1
tau of q2
tau of q8
tau of q4
tau of q5
tau of q6
tau of q7
Adding frictions, motor rotor inertia and springs...
Calculating gregressor...
('input_vars', (q1, q2, q8, q4, q5, q6, q7, dq1, dq2, dq8, dq4, dq5, dq6, dq7, ddq1, ddq2, ddq8, ddq4, ddq5, ddq6, ddq7))
Calculating M, C and G...
Calculating base parameter...
[ 79  82  80  66  69  67  81  53  83  85  84  72  56  71  78  54  59  58
  70  40  45  46  43  41  47  32  57 111 110  14  68  92  44  42  37  15
  17  30  93  16  55  97  98  18  50  29  76  63  89  11  24  38 102  31
  9

In [78]:
robot_def.bary_params

In [79]:
sympy.Matrix(dyn.base_param)

⎡                                                                         1.0⋅
⎢                                                                             
⎢                                                                         1.0⋅
⎢                                                                             
⎢                                                                         1.0⋅
⎢                                                                             
⎢                                                                         1.0⋅
⎢                                                                             
⎢                                                                         1.0⋅
⎢                                                                             
⎢                                                                         1.0⋅
⎢                                                                             
⎢                                                   

In [80]:
from data import RobotModel

# Data to be saved
robot_model = RobotModel(dyn)

# Save
save_data(model_folder, model_name, robot_model)
print('Saved {} parameters'.format(len(robot_model.base_param)))

('Yes DONE', 69)


---
# 3 - Excitation trajectory optimization

In [56]:
# model_name = 'test_psm_long2'
# model_name ='test'
# model_name = 'mtm_3links'
model_name = 'mtm'
#model_name = 'mtm_3links_parallel'
# model_name = 'mtm_4links_parallel'

Load Data

In [57]:
from utils import load_data
model_folder = 'data/' + model_name + '/model/'
robot_model = load_data(model_folder, model_name)

## Create trajectory optimizer object, with H matrix, foourier order, base frequency, joint constraints and cartisian constraints as input

In [58]:
#trajectory_name = 'one'
trajectory_name = 'two_normalized'

In [63]:
from numpy import deg2rad

optimal_trajectory_folder = 'data/optimal_trajectory/'
trajectory_folder = 'data/' + model_name +'/optimal_trajectory/'

base_freq = 0.1
fourier_order = 6

joint_constraints = []
cartesian_constraints = []

if model_name == 'mtm':
#     joint_constraints = [(q1, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q2, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q4, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q5, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q6, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q7, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q8, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi)]
#     joint_constraints = [(q1, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q2, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q4, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q5, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
#                          (q8, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi)]

    joint_constraints = [(q1, deg2rad(-57), deg2rad(29), deg2rad(-160), deg2rad(160)),
                         (q2, deg2rad(-10), deg2rad(60), deg2rad(-180), deg2rad(180)),
                         (q3, deg2rad(-30), deg2rad(30), deg2rad(-180), deg2rad(180)),
                         (q4, deg2rad(-90), deg2rad(195), -6*np.pi, 6*np.pi),
                         (q5, deg2rad(-87), deg2rad(180), -6*np.pi, 6*np.pi),
                         (q6, deg2rad(-40), deg2rad(38), -6*np.pi, 6*np.pi),
                         (q7, deg2rad(-460), deg2rad(450), -10*np.pi, 10*np.pi),
                         (q8, deg2rad(-9), deg2rad(39), deg2rad(-150), deg2rad(150))]

#     cartesian_constraints = [(3,'max',1,0.4,1),
#                              (3,'min',-1,-1,-1)]
elif model_name == 'mtm_3links':
    joint_constraints = [(q1, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
                         (q2, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi),
                         (q3, -np.pi/2, np.pi/2, -2*np.pi, 2*np.pi)]
    
    cartesian_constraints = [(1,'max',1,0.4,1),
                             (1,'min',-1,-1,-1),
                             (2,'max',1,0.4,1),
                             (2,'min',-1,-1,-1)]
    
    # MTM STUFF
#jnt_cnst = [(q1, -0.698132, 1.13446, -2, 2),(q2, -0.261799, 0.872665, -2, 2), (q3,-0.172665, 0.610865, -2, 2)]
#jnt_cnst = [(q1, -1.5, 1.5, -2, 2),(q2, -1.5, 1.5, -2, 2), (q3,-1.5, 1.5, -2, 2)]    

#crt_cnst = [(4, 'max', -0.283, 0.279, -0.25137), (4, 'min', -0.5, -0.321, -0.39137)]
#crt_cnst = [(3, 'max', -0.300, 0.3, -0.26137), (3, 'min', -0.5, -0.45, -0.53137)]
elif model_name == 'mtm_3links_parallel' or model_name == 'mtm_3links_parallel_offset':
    joint_constraints = [(q1, deg2rad(-60), deg2rad(32), deg2rad(-180), deg2rad(180)),
                         (q2, deg2rad(-12), deg2rad(62), deg2rad(-180), deg2rad(180)),
                         (q3, deg2rad(-32), deg2rad(32), deg2rad(-180), deg2rad(180)),
                         (q4, deg2rad(-9), deg2rad(39), deg2rad(-360), deg2rad(360))]
elif model_name == 'mtm_4links_parallel':
    joint_constraints = [(q1, deg2rad(-60), deg2rad(32), deg2rad(-180), deg2rad(180)),
                         (q2, deg2rad(-12), deg2rad(62), deg2rad(-180), deg2rad(180)),
                         (q3, deg2rad(-32), deg2rad(32), deg2rad(-180), deg2rad(180)),
                         (q4, deg2rad(-90), deg2rad(195), -2*np.pi, 2*np.pi),
                         (q5, deg2rad(-9), deg2rad(39), deg2rad(-360), deg2rad(360))]
elif model_name == 'psm':
    pass
elif model_name == 'test_psm_long2':
    joint_constraints = [(q1, -1.4, 1.4, -2, 2),
                         (q2, -0.5, 0.8, -2, 2),
                         (q3,0.05,0.23,-0.4,0.4),
                         (q4,-1.5,1.5,-0.4,0.4),
                         (q5,-1.5,1.5,-0.4,0.4), 
                         (q6,-1.5,1.5,-0.4,0.4),
                         (q7,0.05, 0.75,-0.4,0.4)]
else:
    raise Exception("No robot name matches " + model_name)
    
traj_optimizer = TrajOptimizer(robot_model, fourier_order, base_freq,
                               joint_constraints=joint_constraints,
                               cartesian_constraints = cartesian_constraints)
                                                                                                 
                                                                                                                     
traj_optimizer.optimize()


joint constraint number: 8
cartesian constraint number: 0
constraint number: 32
frames_constrained: []
cond: 63.2305424823
cond: 63.2305424823
cond: 63.2305438651
cond: 63.2305454206
cond: 63.2305467273
cond: 63.2305476855
cond: 63.2305475507
cond: 63.2305474301
cond: 63.2305469102
cond: 63.230546831
cond: 63.2305462355
cond: 63.2305465657
cond: 63.2305455498
cond: 63.2305452507
cond: 63.2305426022
cond: 63.2305436968
cond: 63.2305435924
cond: 63.2305422685
cond: 63.230542147
cond: 63.2305448243
cond: 63.2305435903
cond: 63.2305430356
cond: 63.2305421349
cond: 63.2305404632
cond: 63.2305423457
cond: 63.2305403341
cond: 63.2305407481
cond: 63.2305455446
cond: 63.2305271549
cond: 63.2305453557
cond: 63.2305508891
cond: 63.2305468514
cond: 63.2305379476
cond: 63.2305371954
cond: 63.2305273593
cond: 63.2305324055
cond: 63.2305395266
cond: 63.2305376845
cond: 63.2305363174
cond: 63.230536916
cond: 63.2305409468
cond: 63.230555982
cond: 63.230553682
cond: 63.2305518374
cond: 63.2305496538
co

cond: 57.1165109475
cond: 57.1165055451
cond: 57.1164990842
cond: 57.1164899283
cond: 57.116484739
cond: 57.1164921801
cond: 57.1164942227
cond: 57.1164946073
cond: 57.1164937602
cond: 57.1164930014
cond: 57.1164921785
cond: 57.1164924838
cond: 57.1164931163
cond: 57.1164924952
cond: 57.1164927964
cond: 57.1164922029
cond: 57.1164930421
cond: 57.1164929272
cond: 57.1164941773
cond: 57.116491791
cond: 57.1164924679
cond: 57.1164936865
cond: 57.1164925202
cond: 57.116492659
cond: 57.1164929185
cond: 57.116491982
cond: 57.1164918285
cond: 57.1164918972
cond: 57.1164931257
cond: 57.1164920685
cond: 57.1164919157
cond: 57.1164929853
cond: 57.1164898809
cond: 57.116492822
cond: 57.1164913664
cond: 57.1164924176
cond: 57.1164923491
cond: 57.1164922404
cond: 57.1164937205
cond: 57.1164920637
cond: 57.1164916316
cond: 57.1164921393
cond: 57.1164916744
cond: 57.1164917491
cond: 57.1164922974
cond: 57.1164929348
cond: 57.1164924201
cond: 57.1164926533
cond: 57.1164924904
cond: 57.1164923042
cond:

cond: 352.265881354
cond: 352.265882489
cond: 352.265882562
cond: 352.265877995
cond: 352.265880337
cond: 352.265879622
cond: 352.26588246
cond: 352.265885335
cond: 352.265885221
cond: 352.265879624
cond: 352.26588165
cond: 352.265886198
cond: 352.265880736
cond: 352.26588033
cond: 352.265881337
cond: 304.373997883
cond: 304.373997883
cond: 304.37399337
cond: 304.373955372
cond: 304.374002369
cond: 304.374012857
cond: 304.373885474
cond: 304.373906505
cond: 304.373954923
cond: 304.373941713
cond: 304.374006287
cond: 304.373913013
cond: 304.373931367
cond: 304.374039065
cond: 304.374069159
cond: 304.373999249
cond: 304.374175324
cond: 304.374099696
cond: 304.373967949
cond: 304.374024246
cond: 304.373962755
cond: 304.373691302
cond: 304.373656616
cond: 304.373823759
cond: 304.374007325
cond: 304.374070184
cond: 304.373909415
cond: 304.374116303
cond: 304.374251218
cond: 304.373832167
cond: 304.37390891
cond: 304.374076128
cond: 304.373955592
cond: 304.374055059
cond: 304.374258309
cond:

cond: 269.102094002
cond: 269.102265283
cond: 269.102388465
cond: 269.102437298
cond: 269.102324119
cond: 269.102490043
cond: 269.102530205
cond: 269.102234358
cond: 269.102307167
cond: 269.102459531
cond: 269.102346812
cond: 269.102438234
cond: 269.102714908
cond: 269.102598833
cond: 269.102459742
cond: 269.102435881
cond: 269.102324934
cond: 269.10239153
cond: 269.10239178
cond: 269.102389962
cond: 269.102397296
cond: 269.102392401
cond: 269.102393145
cond: 269.102391107
cond: 269.102393041
cond: 269.102384626
cond: 269.102393186
cond: 269.102390505
cond: 269.102391172
cond: 269.102392102
cond: 269.102393248
cond: 269.102393519
cond: 269.102393121
cond: 269.102393607
cond: 269.102392728
cond: 269.102392227
cond: 269.102392547
cond: 269.102392395
cond: 269.102394198
cond: 269.102390527
cond: 269.102392528
cond: 269.102392494
cond: 269.102393291
cond: 269.102392026
cond: 269.102392165
cond: 269.102392162
cond: 269.102391898
cond: 269.102392041
cond: 269.102393091
cond: 269.102392776
co

cond: 265.132136792
cond: 265.132131067
cond: 265.132130163
cond: 265.132134929
cond: 265.132136193
cond: 265.132136367
cond: 265.132136696
cond: 265.132120862
cond: 265.132138938
cond: 265.132134726
cond: 265.132132694
cond: 265.132135269
cond: 265.132133841
cond: 265.132142481
cond: 265.132135437
cond: 265.132135141
cond: 265.132135098
cond: 265.132137362
cond: 265.132135029
cond: 265.13213522
cond: 265.132135174
cond: 265.132134672
cond: 265.132135117
cond: 265.132134825
cond: 265.13213531
cond: 265.132135328
cond: 265.13213591
cond: 265.132135544
cond: 265.132134957
cond: 265.132135158
cond: 265.132135405
cond: 265.132135125
cond: 270.967971607
cond: 268.721281692
cond: 266.154023325
cond: 264.915690792
cond: 264.915690792
cond: 264.915687547
cond: 264.915670562
cond: 264.915676856
cond: 264.915691212
cond: 264.915611481
cond: 264.915612674
cond: 264.915656142
cond: 264.915639049
cond: 264.915663527
cond: 264.915640007
cond: 264.915626769
cond: 264.915713895
cond: 264.915737469
con

...

cond: 256.602328105
cond: 256.602328002
cond: 256.602327731
cond: 256.602327918
cond: 256.602327921
cond: 256.602327836
cond: 256.60232768
cond: 256.602327967
cond: 256.602327985
cond: 256.602327893
cond: 256.602328053
cond: 256.602328054
cond: 256.602327766
cond: 256.602328014
cond: 256.602327861
cond: 256.60232793
cond: 259.539693638
cond: 256.098266582
cond: 256.098266582
cond: 256.098276377
cond: 256.098252751
cond: 256.098254743
cond: 256.098273303
cond: 256.098192868
cond: 256.098192456
cond: 256.098241374
cond: 256.098215541
cond: 256.098247888
cond: 256.098216603
cond: 256.098203851
cond: 256.098293502
cond: 256.098300278
cond: 256.098219425
cond: 256.098460792
cond: 256.098347169
cond: 256.09823741
cond: 256.098297019
cond: 256.098241953
cond: 256.097959813
cond: 256.09797167
cond: 256.098173215
cond: 256.098260731
cond: 256.098280675
cond: 256.098168137
cond: 256.098354997
cond: 256.098399734
cond: 256.098096421
cond: 256.098197515
cond: 256.098344917
cond: 256.09821781
cond:

cond: 255.196670615
cond: 255.196688279
cond: 255.19657862
cond: 255.196757323
cond: 255.196811044
cond: 255.196512852
cond: 255.196610911
cond: 255.196758978
cond: 255.196631841
cond: 255.196714767
cond: 255.196931807
cond: 255.196867121
cond: 255.196712351
cond: 255.196754173
cond: 255.196658546
cond: 255.196709184
cond: 255.196677759
cond: 255.196675723
cond: 255.196677654
cond: 255.196678792
cond: 255.196678049
cond: 255.196677715
cond: 255.196677935
cond: 255.19667875
cond: 255.196680364
cond: 255.19667802
cond: 255.196677148
cond: 255.196678722
cond: 255.196678768
cond: 255.19667821
cond: 255.196676644
cond: 255.196678176
cond: 255.196678012
cond: 255.196678573
cond: 255.196677525
cond: 255.196678281
cond: 255.196678159
cond: 255.196677568
cond: 255.196678441
cond: 255.196678371
cond: 255.196678216
cond: 255.196678298
cond: 255.196678411
cond: 255.196675433
cond: 255.196677719
cond: 255.196679054
cond: 255.196678314
cond: 255.196678646
cond: 255.19667856
cond: 255.196677061
cond:

cond: 253.989389188
cond: 253.989389019
cond: 253.98938896
cond: 253.989384878
cond: 253.989388248
cond: 253.989389519
cond: 253.989390297
cond: 253.989389628
cond: 253.989389751
cond: 253.989387651
cond: 253.989386392
cond: 253.989390219
cond: 253.989389559
cond: 253.989388346
cond: 253.989389184
cond: 253.989388967
cond: 253.989389064
cond: 253.989388888
cond: 253.989388881
cond: 253.98938898
cond: 253.989388972
cond: 253.98938897
cond: 253.989388887
cond: 253.98938903
cond: 253.989388901
cond: 253.989388986
cond: 253.989388898
cond: 253.989388939
cond: 253.862188404
cond: 253.557912717
cond: 253.557912717
cond: 253.557931595
cond: 253.557900604
cond: 253.557901881
cond: 253.55792722
cond: 253.557842806
cond: 253.557834907
cond: 253.557892418
cond: 253.557859755
cond: 253.557901726
cond: 253.557869205
cond: 253.557854238
cond: 253.557942887
cond: 253.557932593
cond: 253.557860657
cond: 253.558098641
cond: 253.558002044
cond: 253.557884098
cond: 253.55795316
cond: 253.557888494
cond: 

cond: 243.408904825
cond: 243.408900818
cond: 243.408865947
cond: 243.409042145
cond: 243.40896657
cond: 243.408868019
cond: 243.408921611
cond: 243.408863282
cond: 243.408589485
cond: 243.408609899
cond: 243.408799239
cond: 243.408912674
cond: 243.408894809
cond: 243.408783533
cond: 243.408971714
cond: 243.408853143
cond: 243.408722986
cond: 243.408834866
cond: 243.408964026
cond: 243.408849961
cond: 243.408906596
cond: 243.409038927
cond: 243.409045297
cond: 243.408948707
cond: 243.408931648
cond: 243.408851462
cond: 243.408931042
cond: 243.40888921
cond: 243.408888324
cond: 243.408886645
cond: 243.40888906
cond: 243.408881256
cond: 243.408886554
cond: 243.408886104
cond: 243.408892291
cond: 243.408870535
cond: 243.408889608
cond: 243.408891041
cond: 243.408881771
cond: 243.408886502
cond: 243.408884333
cond: 243.408891555
cond: 243.408880477
cond: 243.408881936
cond: 243.40888399
cond: 243.408884925
cond: 243.408881878
cond: 243.408894534
cond: 243.408885352
cond: 243.408878605
cond

cond: 234.573223612
cond: 234.573230165
cond: 234.573229804
cond: 234.573227921
cond: 234.573229062
cond: 234.573228585
cond: 234.573226069
cond: 234.573229137
cond: 234.573229696
cond: 234.573229162
cond: 234.573229105
cond: 234.573229199
cond: 234.573229004
cond: 234.573228776
cond: 234.57323107
cond: 234.573229467
cond: 234.573231451
cond: 234.573231115
cond: 234.573230392
cond: 234.573228327
cond: 234.573228402
cond: 234.573231037
cond: 234.573231765
cond: 234.573228823
cond: 234.573230874
cond: 234.573229326
cond: 234.573229287
cond: 234.573228839
cond: 234.573228933
cond: 234.573229118
cond: 234.573229528
cond: 234.573229474
cond: 234.573228676
cond: 234.573228932
cond: 234.573229582
cond: 234.573229162
cond: 234.573229216
cond: 234.573229245
cond: 240.70063808
cond: 240.066296054
cond: 236.01718428
cond: 236.676631072
cond: 234.565456084
cond: 234.565456084
cond: 234.565477091
cond: 234.565471052
cond: 234.565472954
cond: 234.565461937
cond: 234.565378979
cond: 234.565372161
con

In [64]:
reg_norm_mat = traj_optimizer.calc_normalize_mat()

## Plot optimal excitation trajectory

In [65]:
traj_optimizer.calc_frame_traj()

In [66]:
traj_plotter = TrajPlotter(traj_optimizer.fourier_traj,traj_optimizer.frame_traj,
                           traj_optimizer.const_frame_ind, robot_model.coordinates)
traj_plotter.plot_desired_traj(traj_optimizer.x_result)

traj_plotter.plot_frame_traj(True)

## Save trajectory for later use

In [67]:
dof_order_bf_x_norm = (traj_optimizer.fourier_traj.dof, fourier_order,
                       base_freq, traj_optimizer.x_result, reg_norm_mat)
save_data(trajectory_folder, trajectory_name, dof_order_bf_x_norm)

freq = 500
tf = 35 # including 5 seconds' prestable time
traj_optimizer.make_traj_csv(trajectory_folder, trajectory_name, freq, tf)

In [68]:
from IPython.display import HTML, display
import tabulate

table = []
table.append(["joint", 'qo'] +
             ["a"+str(i+1) for i in range(fourier_order)] +
             ["b"+str(i+1) for i in range(fourier_order)])
for i in range(traj_optimizer.fourier_traj.dof):
    line = []
    line.append(robot_model.coordinates[i])
    line += np.round(traj_optimizer.x_result[i*(1 + fourier_order*2): (i+1)*(1 + fourier_order*2)], 4).tolist()
    table.append(line)

display(HTML(tabulate.tabulate(table, tablefmt='html')))

joint,qo,a1,a2,a3,a4,a5,a6,b1,b2,b3,b4,b5,b6
q1,-0.3338,-0.0281,0.0566,0.0115,-0.4216,1.3938,1.7039,-0.0996,0.1164,0.0081,-0.1999,0.2456,-0.2141
q2,0.3413,-0.0218,-0.2995,-0.2413,-0.2218,-0.3036,0.2819,0.0013,0.3679,0.0664,-0.3803,-0.1107,0.9744
q8,0.2848,0.0055,0.0715,0.1158,-0.6311,0.0616,-0.2653,-0.0642,0.0218,-0.0954,-0.4457,0.2007,0.2664
q4,1.0711,-0.4967,-1.0293,0.8883,-1.8455,-1.5918,2.4066,0.0873,0.8079,0.416,0.6947,-0.7217,-2.3011
q5,0.5132,0.3703,-0.8054,-0.8411,0.9688,0.8021,2.2541,-0.0895,1.1835,1.1867,-0.5559,-0.6951,-2.5493
q6,0.1815,0.046,0.269,0.0926,-0.4523,-0.0681,0.6493,0.0768,-0.1543,0.0807,0.2341,-0.042,0.0182
q7,1.4221,1.4724,0.0368,6.1365,3.2479,-0.0121,-5.403,-0.5177,1.8184,-6.8554,-1.9775,-2.4938,5.9238


---
# Data processing

In [4]:
from identification import load_trajectory_data, diff_and_filt_data, plot_trajectory_data, plot_meas_pred_tau, gen_regressor

## Load data

In [6]:
# Names as Arguments
# model_name = 'test_psm_long'
# model_name = 'mtm_3links'
model_name = 'mtm'
# model_name == 'mtm_3links_parallel'
# model_name == 'mtm_4links_parallel'

# trajectory_name = 'one'
# trajectory_name = 'two'
trajectory_name = 'two_normalized'

In [31]:
# Folders Standard
model_folder = 'data/' + model_name + '/model/'
trajectory_folder = 'data/' + model_name +'/optimal_trajectory/'
results_folder = 'data/' + model_name +'/measured_trajectory/'
results_data_file = results_folder + trajectory_name + '_results.csv'

# Load Results
dof, fourier_order, base_freq, traj_optimizer_result, reg_norm_mat = load_data(trajectory_folder, trajectory_name)


trajectory_sampling_rate = 500
#t, q_raw, tau_raw = load_trajectory_data(results_data_file, trajectory_sampling_rate)
t, q_raw, dq_raw, tau_raw = load_trajectory_data(results_data_file, trajectory_sampling_rate)
dof = q_raw.shape[1]

# Load Robot Model
robot_model = load_model_data(model_folder,model_name)

ValueError: need more than 4 values to unpack

## Calculate filter cut-off frequency

In [33]:
fc_mult = 4.0
# this should be loaded from previously saved data
wf = 0.1
order = 6
fc = wf * order * fc_mult
print("cut frequency is {}".format(fc))

cut frequency is 2.4


## Get velocity and acceleration data and filter raw data

In [34]:
t_cut, q_f, dq_f, ddq_f, tau_f, q_raw_cut, tau_raw_cut = diff_and_filt_data(dof, 1.0/trajectory_sampling_rate, t,
                                                                        q_raw, dq_raw, tau_raw, fc, fc, fc, fc)
plot_trajectory_data(t_cut, q_raw_cut, q_f, dq_f, ddq_f, tau_raw_cut, tau_f)

q_raw shape: (14989, 7)
0
1
2
3
4
5
6
plot shape: 470


---
# Regression

## Ordinary Least Square (OLS)

### Generate regressor matrix for base parameters

In [35]:
base_param_num = robot_model.base_num
H_b_func = robot_model.H_b_func
W_b, tau_s = gen_regressor(base_param_num, H_b_func, q_f, dq_f, ddq_f, tau_f)

In [36]:
print(W_b.shape, tau_s.shape)
print np.linalg.cond(W_b)

((102123, 69), (102123,))
8166.481157839068


In [37]:
xb_ols = np.linalg.lstsq(W_b, tau_s)[0]

/home/yan/anaconda2/envs/dynamics/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


In [38]:
#sympy.Matrix.hstack(sympy.Matrix(robot_model.base_param), sympy.Matrix(xb_ols))

from IPython.display import HTML, display
import tabulate

table = [["Base Parameter", "Value"]]

for i in range(robot_model.base_num):
    param_str = str(sympy.Matrix(robot_model.base_param)[i])
    max_disp_len = 50
    line = [param_str if len(param_str) <= max_disp_len else param_str[:max_disp_len] + '...', xb_ols[i]]
    #['%.7s ...'%b if len(str(b)) > 7 else str(b)
    table.append(line)

display(HTML(tabulate.tabulate(table, tablefmt='html')))

Base Parameter,Value
1.0*L7xy,2.3021815850897337e-05
1.0*L7xz,6.694537753920617e-05
1.0*L7yz,-4.792865145582053e-05
1.0*L6xy,-0.0008239221279158822
1.0*L6yz,0.00046701678493795556
1.0*L6xz,8.94858820072687e-05
1.0*L6xx - 1.0*L6yy + 1.0*L7yy,0.0005258140947768681
1.0*L5xy,-0.0006103015075525058
1.0*L7zz,-5.7605395689497296e-05
1.0*l7y,-0.00021924622551982133


## Compare measured torque and predicted torque

### Generate predicted torque

In [39]:
tau_p = np.zeros(tau_f.shape)
tau_ps = W_b.dot(xb_ols)
for i in range(dof):
    tau_p[:, i] = tau_ps[i::dof]
    

### Evaluate regression

In [40]:
var_regression_error_ols = np.linalg.norm(tau_ps - tau_s) / (tau_ps.size - base_param_num)
print("variance of regression error using OLS:")
print(var_regression_error_ols)

std_dev_xb_ols = np.sqrt(np.diag(var_regression_error_ols * np.linalg.inv(W_b.transpose().dot(W_b))))
print("standard deviation of xb using OLS:")
print(std_dev_xb_ols)

pct_std_dev_xb_ols = std_dev_xb_ols / np.abs(xb_ols)
print("percentage of standard deviation of xb using OLS: ")
print(pct_std_dev_xb_ols)

variance of regression error using OLS:
0.00041535309168010755
standard deviation of xb using OLS:
[1.52345019e-06 1.02269065e-06 1.03057772e-06 9.02911987e-06
 8.87326572e-06 7.89711779e-06 1.92036655e-05 1.12741078e-05
 2.96891378e-06 3.46428143e-06 3.42720796e-06 2.25605470e-05
 1.03749637e-05 3.43495904e-05 1.89959293e-05 1.09561691e-05
 3.70711704e-05 1.33855254e-05 2.90083901e-05 2.49691466e-05
 1.85869138e-05 2.12475252e-05 3.27964242e-05 1.61486114e-05
 7.50011349e-04 2.32521402e-03 3.07967306e-04 1.26233559e-04
 1.91194251e-05 1.12552486e-04 3.10167342e-04 2.02488613e-05
 2.27366849e-05 2.47316853e-04 2.96680500e-04 7.23722398e-05
 4.77990351e-05 1.05986750e-04 4.47299615e-05 4.86463882e-04
 2.43866784e-04 8.07760475e-04 7.36862438e-04 3.00927357e-04
 5.29444434e-04 8.91534430e-05 4.17219057e-04 7.70584748e-05
 3.49965958e-05 9.01274170e-05 5.08290601e-04 3.35841100e-04
 3.40976532e-03 5.37466749e-04 1.86197574e-04 1.69075621e-04
 1.75182884e-04 1.76388635e-04 1.69112072e-04 2

### Plot measured torque and predicted torque

In [41]:
plot_data_num = int(1 / wf * trajectory_sampling_rate)
plot_meas_pred_tau(t_cut[:plot_data_num], tau_f[:plot_data_num, :], tau_p[:plot_data_num, :])

## Weighted Least Square (WLS)

In [42]:
np.sqrt(np.linalg.norm(tau_f - tau_p, axis=0)/(tau_f.shape[0] - base_param_num))

array([0.02768482, 0.04699249, 0.04054076, 0.02216207, 0.01782177,
       0.01096289, 0.00486901])

In [43]:
weight = np.max(tau_f, axis=0) - np.min(tau_f, axis=0)
weight
weight[1] /= 1
weight[2] /= 1
weight[3] /= 1
weight

array([3.67977947, 5.15810573, 3.33847912, 0.4932939 , 0.21108453,
       0.09150989, 0.00616569])

In [44]:
W_b.shape, tau_s.shape

In [45]:
# repeat the weight to generate a large vecoter for all the data
weights = 1.0/np.tile(weight, W_b.shape[0]/weight.shape[0])

In [46]:
weights.shape

In [47]:
W_b_wls = np.multiply(W_b, np.asmatrix(weights).transpose())
tau_s_wls = np.multiply(tau_s, weights)

In [48]:
xb_wls = np.linalg.lstsq(W_b_wls, tau_s_wls)[0]

/home/yan/anaconda2/envs/dynamics/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


In [49]:
sympy.Matrix.hstack(sympy.Matrix(robot_model.base_param), sympy.Matrix(xb_wls))

⎡                                                                         1.0⋅
⎢                                                                             
⎢                                                                         1.0⋅
⎢                                                                             
⎢                                                                         1.0⋅
⎢                                                                             
⎢                                                                         1.0⋅
⎢                                                                             
⎢                                                                         1.0⋅
⎢                                                                             
⎢                                                                         1.0⋅
⎢                                                                             
⎢                                                   

In [50]:
tau_p_wls = np.zeros(tau_f.shape)
tau_ps_wls = W_b.dot(xb_wls)
for i in range(dof):
    tau_p_wls[:, i] = tau_ps_wls[i::dof]

In [53]:
plot_meas_pred_tau(t_cut[:plot_data_num], tau_f[:plot_data_num, :], tau_p_wls[:plot_data_num, :])

In [52]:
np.linalg.norm(tau_f[:plot_data_num, :] - tau_p_wls[:plot_data_num, :], axis=0)\
/ np.linalg.norm(tau_f[:plot_data_num, :], axis=0)

array([0.10860799, 0.33826741, 0.21083488, 0.38502505, 0.57332539,
       0.20940031, 0.22822484])

## Convex optimization

### Generate regressor matrix for barycentric parameters

In [149]:
from identification import SDPOpt

bary_param_num = len(robot_model.bary_param)
H_func = robot_model.H_func
W, tau_s = gen_regressor(bary_param_num, H_func, q_f, dq_f, ddq_f, tau_f)

In [150]:
W_cvx = np.multiply(W, np.asmatrix(weights).transpose())
tau_s_cvx = np.multiply(tau_s, weights)

In [151]:
len(robot_model.std_param)

| min mass | max mass | min $r_x$ | max $r_x$ | min $r_y$ | max $r_y$ | min $r_z$ | max $r_z$ | max $F_c$ | max $F_v$ | max $F_o$ |
|----------|----------|-----------|-----------|-----------|-----------|-----------|-----------|-----------|--------------|-----------|

In [173]:
sdp_constraints = []
if model_name == 'mtm':
    sdp_constraints = [(0.5, 10, -0.1, 0.1, -0.1, 0.1, -0.1, 0.3, 0.2, 0.2, 0.2),
                      (0.3, 5, -0.1, 0.2, -0.05, 0.05, -0.05, 0.05, 0.2, 0.2, 0.2),
                      (0.1, 3, -0.0, 0.3, -0.02, 0.02, -0.01, 0.01, 0.2, 0.2, 0.2),
                      (0.1, 1, -0.01, 0.01, -0.0, 0.1, -0.1, 0.0, 0.2, 0.2, 0.1),
                      (0.02, 0.3, -0.01, 0.01, -0.05, 0.0, -0.05, 0.0, 0.1, 0.1, 0.1),
                      (0.02, 0.2, -0.005, 0.005, -0.00, 0.05, -0.05, 0.0, 0.1, 0.1, 0.1),
                      (0.01, 0.1, -0.000, 0.000, -0.00, 0.00, -0.00, 0.05, 0.1, 0.1, 0.1),
                      (0.3, 5, -0.10, 0.05, -0.05, 0.05, -0.05, 0.05, 0.1, 0.2, 0.2),
                      (0.05, 0.5, 0.10, 0.15, -0.00, 0.00, 0.025, 0.03, 0.1, 0.1, 0.1),
                      (0.0, 0, 0.00, 0.00, -0.00, 0.00, 0.000, 0.00, 0.1, 0.1, 0.0)]
elif model_name == 'mtm_3links':
    sdp_constraints = [(0.5, 10, -0.1, 0.1, -0.1, 0.1, -0.1, 0.3, 0.5, 0.5, 0.5),
                      (0.3, 5, -0.1, 0.2, -0.05, 0.05, -0.05, 0.05, 0.5, 0.5, 0.5),
                      (0.1, 3, -0.0, 0.3, -0.02, 0.02, -0.01, 0.01, 0.1, 0.5, 0.5),
                      (0.1, 1, -0.01, 0.01, -0.0, 0.1, -0.1, 0.0, 0.1, 0.1, 0.1),
                      (0.02, 0.3, -0.01, 0.01, -0.05, 0.0, -0.05, 0.0, 0.1, 0.1, 0.1),
                      (0.02, 0.2, -0.005, 0.005, -0.00, 0.05, -0.05, 0.0, 0.1, 0.1, 0.1),
                      (0.01, 0.1, -0.000, 0.000, -0.00, 0.00, -0.00, 0.05, 0.1, 0.1, 0.1),
                      (0.3, 5, -0.10, 0.05, -0.05, 0.05, -0.05, 0.05, 0.5, 0.5, 0.5),
                      (0.3, 5, 0.10, 0.15, -0.00, 0.00, 0.025, 0.03, 0.1, 0.1, 0.1),
                      (0.3, 5, 0.10, 0.15, -0.00, 0.00, 0.025, 0.03, 0.1, 0.1, 0.1)]
else:
    raise Exception("No robot name matches " + model_name)

In [174]:


sdp_opt_std = SDPOpt(W_cvx, tau_s_cvx, robot_model, sdp_constraints)
# sdp_opt_std = SDPOpt(W, tau_s, robot_model, sdp_constraints)
%time sdp_opt_std.solve()

Regressor shape: (102123, 120)
Regressand shape: (102123,)
Creating variables...
Creating optimization objective...
Creating constraints...
Solving problem...
----------------------------------------------------------------------------
	SCS v1.2.6 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2016
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 4268296, CG tol ~ 1/iter^(2.00)
eps = 1.00e-03, alpha = 1.50, max_iters = 10000, normalize = 1, scale = 1.00
Variables n = 121, constraints m = 102447
Cones:	linear vars: 133
	soc vars: 102125, soc blks: 1
	sd vars: 189, sd blks: 9
Setup time: 1.62e-01s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf      -nan      -inf       inf       inf  3.50

  9400| 3.10e-01  1.06e-02  5.74e-04  1.25e+02  1.26e+02  5.87e-14  1.63e+03 
  9500| 3.10e-01  1.06e-02  5.65e-04  1.27e+02  1.27e+02  5.98e-14  1.65e+03 
  9600| 3.09e-01  1.00e-02  5.54e-04  1.28e+02  1.28e+02  6.08e-14  1.67e+03 
  9700| 3.09e-01  1.02e-02  5.43e-04  1.29e+02  1.29e+02  6.19e-14  1.69e+03 
  9800| 3.08e-01  1.05e-02  5.35e-04  1.30e+02  1.30e+02  6.30e-14  1.70e+03 
  9900| 3.08e-01  9.87e-03  5.24e-04  1.31e+02  1.32e+02  6.40e-14  1.72e+03 
 10000| 3.08e-01  9.73e-03  5.15e-04  1.33e+02  1.33e+02  6.51e-14  1.74e+03 
----------------------------------------------------------------------------
Status: Solved/Inaccurate
Hit max_iters, solution may be inaccurate
Timing: Solve time: 1.74e+03s
	Lin-sys: avg # CG iterations: 35.15, avg solve time: 1.72e-01s
	Cones: avg projection time: 2.59e-04s
----------------------------------------------------------------------------
Error metrics:
dist(s, K) = 6.4829e-09, dist(y, K*) = 2.7388e-09, s'y/|s||y| = 2.0676e-11
|Ax + s -

## Compare measured torque and predicted torque

### Generate predicted torque

In [175]:
tau_sdp_p = np.zeros(tau_f.shape)
tau_sdp_ps = W.dot(sdp_opt_std.x_result)
for i in range(dof):
    tau_sdp_p[:, i] = tau_sdp_ps[i::dof]

### Plot measured torque and predicted torque

In [200]:
plot_data_num = int(1 / wf * trajectory_sampling_rate)
plot_meas_pred_tau(t_cut[:plot_data_num] - t_cut[0],
                   tau_f[:plot_data_num, :],
                   tau_sdp_p[:plot_data_num, :])
# plot_meas_pred_tau(t_cut, tau_f, tau_sdp_p)

In [196]:
sympy.Matrix.hstack(sympy.Matrix(robot_model.bary_param), sympy.Matrix(sdp_opt_std.x_result))

⎡L1xx    0.986192165056305  ⎤
⎢                           ⎥
⎢L1xy  -1.38516767370781e-14⎥
⎢                           ⎥
⎢L1xz  -7.76343556708024e-27⎥
⎢                           ⎥
⎢L1yy    0.986192165056345  ⎥
⎢                           ⎥
⎢L1yz  -2.12133368993413e-16⎥
⎢                           ⎥
⎢L1zz   5.58233934249789e-5 ⎥
⎢                           ⎥
⎢l1x   2.40588966703881e-27 ⎥
⎢                           ⎥
⎢l1y   4.76358117877349e-17 ⎥
⎢                           ⎥
⎢l1z     0.144175828054554  ⎥
⎢                           ⎥
⎢ m₁     1.79307211904898   ⎥
⎢                           ⎥
⎢Fc₁    0.0419135562259835  ⎥
⎢                           ⎥
⎢Fv₁    0.0497210098280573  ⎥
⎢                           ⎥
⎢Fo₁    -0.0698865408680767 ⎥
⎢                           ⎥
⎢L2xx   0.0194654973664062  ⎥
⎢                           ⎥
⎢L2xy   0.00876972571924054 ⎥
⎢                           ⎥
⎢L2xz   0.00593865075240333 ⎥
⎢                           ⎥
⎢L2yy   0.0330003696264969  ⎥
⎢         

In [178]:
from identification import barycentric2standard_params

In [179]:
sdp_opt_std.x_result

array([ 9.86192165e-01, -1.38516767e-14, -7.76343557e-27,  9.86192165e-01,
       -2.12133369e-16,  5.58233934e-05,  2.40588967e-27,  4.76358118e-17,
        1.44175828e-01,  1.79307212e+00,  4.19135562e-02,  4.97210098e-02,
       -6.98865409e-02,  1.94654974e-02,  8.76972572e-03,  5.93865075e-03,
        3.30003696e-02,  9.67284032e-04,  5.04838084e-02, -4.02232774e-01,
        6.80708808e-02,  6.43105023e-04,  5.00027870e+00,  1.09124988e-01,
        1.99956998e-01, -2.00003749e-01,  6.19401340e-04,  8.10870769e-04,
        6.31621227e-05,  1.18055051e-03, -3.76610295e-05,  1.71030714e-03,
        4.07898714e-02, -2.79045453e-02, -2.19072695e-03,  1.41404147e+00,
        1.00110498e-03,  9.95052980e-04,  2.00008574e-01,  2.02184077e-02,
        3.45713217e-03,  1.34006236e-03,  1.24674614e-02,  2.77597880e-03,
        1.57090952e-03, -8.13941900e-04,  1.04485762e-02, -1.29595950e-02,
        1.33525052e-01,  4.08774176e-02,  1.22874520e-02, -1.04736499e-04,
        7.97410096e-04, -

In [180]:
x_std = barycentric2standard_params(sdp_opt_std.x_result, robot_model)

In [181]:
sympy.Matrix.hstack(sympy.Matrix(robot_model.std_param), sympy.Matrix(x_std))

⎡I1xx    0.974599396882406  ⎤
⎢                           ⎥
⎢I1xy  -1.38516767370781e-14⎥
⎢                           ⎥
⎢I1xz  -7.56998482337162e-27⎥
⎢                           ⎥
⎢I1yy    0.974599396882446  ⎥
⎢                           ⎥
⎢I1yz  -2.08303108884751e-16⎥
⎢                           ⎥
⎢I1zz   5.58233934249789e-5 ⎥
⎢                           ⎥
⎢r1x   1.34176960395484e-27 ⎥
⎢                           ⎥
⎢r1y   2.65665899779873e-17 ⎥
⎢                           ⎥
⎢r1z    0.0804071551405432  ⎥
⎢                           ⎥
⎢ m₁     1.79307211904898   ⎥
⎢                           ⎥
⎢Fc₁    0.0419135562259835  ⎥
⎢                           ⎥
⎢Fv₁    0.0497210098280573  ⎥
⎢                           ⎥
⎢Fo₁    -0.0698865408680767 ⎥
⎢                           ⎥
⎢I2xx   0.0185387373447571  ⎥
⎢                           ⎥
⎢I2xy   0.00329396309376292 ⎥
⎢                           ⎥
⎢I2xz   0.00588691805246235 ⎥
⎢                           ⎥
⎢I2yy  0.000643849538560864 ⎥
⎢         

In [182]:
sdp_constraints